In [8]:
#dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
#start up browser
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/Users/Madison/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [3]:
#visit NASA Mars News
url1 = "https://redplanetscience.com/"
browser.visit(url1)
#start up parser
html = browser.html
soup = BeautifulSoup(html, "html.parser")
#scrape the first headline and paragraph 
NASA_title = soup.find("div", class_="content_title").get_text() 
NASA_par = soup.find("div", class_="article_teaser_body").get_text() 
print(NASA_title)
print(NASA_par)

From JPL's Mailroom to Mars and Beyond
Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.


In [7]:
#visit JPL Mars Space Images - Featured Image
url2 = "https://spaceimages-mars.com/"
browser.visit(url2)
#start up parser
html = browser.html
soup = BeautifulSoup(html, "html.parser")
#browser.links.find_by_partial_text('FULL IMAGE').contentclick()
#pull the featured image
featured_image = soup.find("img", class_="headerimage fade-in")['src']
featured_image_url = url2 + featured_image
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


In [9]:
#Mars Facts - Table
url3 = "https://galaxyfacts-mars.com/"
mars_tables = pd.read_html(url3)
mars_tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [16]:
#only want the table on Mars
mars_table = mars_tables[1]
mars_table = mars_table.rename(columns = {0:"Fact", 1:"Data"})
html_mars_table = mars_table.to_html()